In [1]:
# Initial Imports
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy

In [28]:
# Read in lyrics data
lyrics = pd.read_csv("data/songdata.csv")

In [4]:
# Required columns
cols=['song', 'artist', 'trackid', 'danceability', 'energy', 'key', 'mode', 
      'speechiness', 'acousticness', 'instrumentalness', 'liveness', 
      'valence', 'tempo', 'duration_ms', 'time_signature',
     'duration', 'analysis_sample_rate', 'analysis_channels', 
      'end_of_fade_in', 'start_of_fade_out', 'loudness', 'tempo', 'tempo_confidence', 
      'time_signature', 'time_signature_confidence', 'key', 'key_confidence', 'mode', 'mode_confidence']

In [5]:
# Creating dataframe for audio features
features=pd.DataFrame(columns=cols)

In [6]:
# Client Credentials
client_id = "5fc0da4106eb48e1913868f4950828fb"
client_secret = "5a549bc4674e47dfaf3b06da4500a44d"

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

for index,row in lyrics.head(1000).iterrows():
    song = row['song']
    
    temp = pd.DataFrame(columns = cols)
    temp['song'] = [song]
    temp['artist'] = [row['artist']]
    
    try:
        result = sp.search(song, type='track', limit=1)

    except error as e:
        print(e)
        continue
        
    if len(result['tracks']['items']) == 0:
        temp['trackid'] = ['NA']
        features = features.append(temp, ignore_index=True)
        continue
    
    trackid=result['tracks']['items'][0]['id']
    temp['trackid'] = [trackid]
    
    try:
        feat=sp.audio_features(trackid)[0]
        analysis=sp.audio_analysis(trackid)['track']
    except error as e:
        print(e)
        continue
    
    combined = {**feat, **analysis}
    
    for col in cols:
        if col in ['song', 'artist', 'trackid']:
            continue
        temp[col]= combined[col]
        
    features = features.append(temp, ignore_index=True)

SpotifyException: http status: 404, code:-1 - https://api.spotify.com/v1/audio-analysis/2TQXlTxll0yrNuNJMXTQk9:
 analysis not found

In [10]:
import nltk
from nltk.tokenize import RegexpTokenizer

all_text = ' '.join(lyrics.text)
all_text = all_text.replace('\n', '')
sent_text = nltk.sent_tokenize(all_text)

In [13]:
tokenizer = RegexpTokenizer(r'\w+').tokenize

sent_tokenized = []
for sent in sent_text:
    sent_tokenized.append(tokenizer(sent))

In [18]:
from gensim.models import Word2Vec
from gensim.test.utils import common_texts, get_tmpfile

path = get_tmpfile("word2vec.model")
model = Word2Vec(sent_tokenized, size=200, window=5, min_count=3, workers=4)

model.save("word2vec.model")

In [31]:
wv = []
for lyr in lyrics.text:
    vec = [0] * 200
    c = 0
    words = tokenizer(lyr)
    for word in words:
        try:
            vec = vec + model[word]
            c = c + 1
        except:
            vec = vec + [0] * 200
    vec = np.array(vec) / c
    wv.append(vec)

/home/aman/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/home/aman/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':


In [65]:
req_cols = list(range(200))
wv_df = pd.DataFrame(wv[:1000])[req_cols]
wv_df['song'] = lyrics['song'][:200]

In [67]:
wv_df.to_csv('word_vectors_songs.csv', index=None)

In [ ]:
# Removing empty rows
features = features[features['trackid'] != 'NA']

In [ ]:
# Making a copy for preservation
features_copy = features.copy()

In [ ]:
# Remove columns for clustering
features = features.reset_index()
cols.remove('song')
cols.remove('artist')
cols.remove('trackid')

In [ ]:
# Kmeans
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10, random_state=0).fit(features[cols])
features['label'] = kmeans.labels_

In [24]:
lyrics.text[0]

"Look at her face, it's a wonderful face  \nAnd it means something special to me  \nLook at the way that she smiles when she sees me  \nHow lucky can one fellow be?  \n  \nShe's just my kind of girl, she makes me feel fine  \nWho could ever believe that she could be mine?  \nShe's just my kind of girl, without her I'm blue  \nAnd if she ever leaves me what could I do, what could I do?  \n  \nAnd when we go for a walk in the park  \nAnd she holds me and squeezes my hand  \nWe'll go on walking for hours and talking  \nAbout all the things that we plan  \n  \nShe's just my kind of girl, she makes me feel fine  \nWho could ever believe that she could be mine?  \nShe's just my kind of girl, without her I'm blue  \nAnd if she ever leaves me what could I do, what could I do?\n\n"